In [ ]:
%load_ext autoreload
%autoreload 2

import os
import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import datetime as dt
import scipy.optimize
import landau
from scipy.interpolate import CubicSpline, interp2d
from scipy.integrate import quad
from scipy.special import erf
from scipy import linalg

import importlib

# local imports
from lib.constants import *

In [ ]:
savedir = "plots_9_13_23/"
dosave = False
pqual = ""

plt.rcParams.update({'font.size': 14})

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/calib-mc/"
# perhit_df = pd.read_hdf(filedir + 'calib_mcnuphase2.df')
perhit_df = pd.read_hdf(filedir + 'calib_mcnuphase1.df')

In [ ]:
# define the fit function
def fit_lossy(x, *p):
    N = p[0]
    sigma = p[1]
    center = p[2]
    beta = p[3]
    m = p[4]
    return N*stats.crystalball.pdf(x, beta, m, center, sigma)

def fit_gaus(x, A, sigma, mu):
    return A*np.exp(-(x-mu)**2 / sigma**2)

In [ ]:
def charge_ratio_plot(ratio, varx, binx, ax, p0mean=1e-2, biny=None):
    if biny is None:
        biny = np.linspace(0, 0.02, 81)
    p0sigma = p0mean/10.

    
    N,_,_,__ = ax.hist2d(varx, ratio, bins=[binx, biny])
    ax.colorbar()

    xcenters = (binx[1:] + binx[:-1]) / 2.
    charge_ratios = (biny[1:] + biny[:-1]) / 2.
    cals = []
    cal_errs = []
    xs = []

    for i in range(len(xcenters)):
        Nfit = N[i, :]
        p0 = [np.max(Nfit), p0sigma, p0mean, 1, 2.]
        
        when_fit = (Nfit > 0) & (charge_ratios > 0.005)
        try:
            popt, perr = curve_fit(fit_lossy, charge_ratios[when_fit], Nfit[when_fit], sigma=np.sqrt(Nfit[when_fit]), 
                                   p0=p0, maxfev=100_000)
            cal = popt[2]
            cal_err = np.sqrt(np.diag(perr))[2]
        except:
            cal = p0mean
            cal_err = np.inf
            
        cal_errs.append(cal_err)
        cals.append(cal)
        xs.append(xcenters[i])
    # ax.plot(xs, cals, color="r")

    # plt.xlabel("Residual Range [cm]")
    # plt.ylabel("Hit Integral / True Charge [ADDC / #elec]")
    return xcenters, np.array(cals), np.array(cal_errs)


In [ ]:
perhit_df["phi"] = np.arccos(np.abs(perhit_df.dir_x))*180/np.pi

In [ ]:
when_plt = (perhit_df.pitch < 0.4) & (perhit_df.phi > 70) & (perhit_df.phi < 85)

In [ ]:
bins = np.linspace(40, 120, 61)
var = perhit_df.true_nelec / perhit_df.integral
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.xlabel("True Charge / Reco. Integral Charge $e^-$/ADC")
plt.ylabel("Entries")

N,_,_ = plt.hist(var[when_plt], bins=bins, label="Entries")

Nfit = N
centers = (bins[1:] + bins[:-1]) / 2.

when_fit = (Nfit > 0)
p0 = [np.max(Nfit), 2, -78, 1, 2.]
fitf = fit_lossy

popt, perr = curve_fit(fitf, -centers[when_fit], Nfit[when_fit], sigma=np.sqrt(Nfit[when_fit]), 
                       p0=p0, maxfev=100_000)
plt.plot(centers, fitf(-centers, *popt), color="r", label="Crystal-Ball Fit")
plt.axvline(-popt[2], color="r", linestyle=":")
plt.text(0.65, 0.5, "True Gain:\n    %.1f $e^-$/ADC" % (-popt[2]), fontsize=14, transform=plt.gca().transAxes)
print(popt)

plt.legend()

plt.tight_layout()

if dosave: plt.savefig(savedir + "integral_calibration.pdf")

In [ ]:
bins = np.linspace(0, 0.02, 101)
var = perhit_df.integral / perhit_df.true_nelec
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.xlabel("True Charge / Reco. Integral Charge $e^-$/ADC")
plt.ylabel("Entries")

N,_,_ = plt.hist(var[when_plt], bins=bins, label="Entries")

Nfit = N
centers = (bins[1:] + bins[:-1]) / 2.

when_fit = (Nfit > 0)
p0 = [np.max(Nfit), 0.005, 0.013, 1, 2.]
fitf = fit_lossy

popt, perr = curve_fit(fitf, centers[when_fit], Nfit[when_fit], sigma=np.sqrt(Nfit[when_fit]), 
                       p0=p0, maxfev=100_000)
plt.plot(centers, fitf(centers, *popt), color="r", label="Crystal-Ball Fit")
plt.axvline(popt[2], color="r", linestyle=":")
plt.text(0.05, 0.5, "True Calo Cosnt:\n    %.5f ADC/$e^-$" % (popt[2]), fontsize=14, transform=plt.gca().transAxes)
print(popt)

plt.legend()

plt.tight_layout()

# if dosave: plt.savefig(savedir + "integral_calibration.pdf")

In [ ]:
1/78.1

In [ ]:
1/0.01263